# Recommender Systems

In this part of the exercise, we implement the collaborative filtering learning algorithm and apply it to a dataset of movie ratings. 
[MovieLens 100k Dataset from GroupLens Research](http://grouplens.org/datasets/movielens)
This dataset consists of ratings on a scale of 1 to 5. The dataset has $ n_u = 943 $ users, and $ n_m = 1682 $ movies. 

In the next parts of this exercise, we will implement the function `cofiCostFunc` that computes the collaborative filtering objective function 
and gradient. After implementing the cost function and gradient, we will use `scipy.minimize` to learn the parameters for collaborative filtering.

## Movie ratings dataset
First, we load the dataset `ex8_movies.mat`, providing the variables `Y` and `R`. 

- The matrix $Y$ is a $(num_{movies} \times num_{users} \space)$ matrix that stores the ratings $ y(i,j) $ (from 1 to 5).
- The matrix $ R $ is a binary-valued indicator matrix, where $ R(i,j) = 1 $ if user $ j $ gave a rating to movie $ i $, and $ R(i,j) = 0 $ otherwise.

The objective of collaborative filtering is to predict movie ratings for the movies that users have not yet rated, that is, 
the entries with $ R(i,j) = 0 $. This will allow us to recommend the movies with the highest predicted ratings to the user.

To help you understand the matrix $ Y $, the script `ex8_cofi` will compute the average movie rating for the first movie (Toy Story) 
and output the average rating to the screen.

Throughout this part of the exercise, you will also be working with the matrices, $ X $ and $ \Theta $:

$$
X = 
\begin{bmatrix}
(x^{(1)})^T \\
(x^{(2)})^T \\
\vdots \\
(x^{(n_m)})^T \\
\end{bmatrix}, \quad 
\Theta = 
\begin{bmatrix}
(\theta^{(1)})^T \\
(\theta^{(2)})^T \\
\vdots \\
(\theta^{(n_u)})^T \\
\end{bmatrix}.
$$

The $ i $-th row of $ X $ corresponds to the feature vector $ x^{(i)} $ for the $ i $-th movie, and the $ j $-th row of $ \Theta $ corresponds to one parameter vector $ \theta^{(j)} $ for the $ j $-th user. Both $ x^{(i)} $ and $ \theta^{(j)} $ are $ n $-dimensional vectors. 

For the purposes of this exercise, you will use $ n = 100 $, and therefore, $ x^{(i)} \in \mathbb{R}^{100} $ and $ \theta^{(j)} \in \mathbb{R}^{100} $. Correspondingly, $ X $ is a $ n_m \times 100 $ matrix and $ \Theta $ is a $ n_u \times 100 $ matrix.


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from utils.util8 import visualizeFit
plt.style.available
plt.style.use('seaborn-v0_8')

In [2]:
data = scipy.io.loadmat('./data/ex8_movies.mat')
Y = data['Y']
R = data['R']
Y.shape, R.shape

((1682, 943), (1682, 943))

In [3]:
print('Average rating for movie 1 (Toy Story):', np.mean(Y[0, R[0, :] == 1]))
np.sum(Y[0])/np.sum(R[0])

Average rating for movie 1 (Toy Story): 3.8783185840707963


3.8783185840707963

## Collaborative Filtering Learning Algorithm

Now, we start implementing the collaborative filtering learning algorithm. 
We start by implementing the cost function (without regularization).

The collaborative filtering algorithm in the setting of movie recommendations considers a set of 
$ n $-dimensional parameter vectors $ x^{(1)}, \ldots, x^{(n_m)} $ and $ \theta^{(1)}, \ldots, \theta^{(n_u)} $, 
where the model predicts the rating for movie $ i $ by user $ j $ as 

$$
y^{(i,j)} = (\theta^{(j)})^T x^{(i)}.
$$

Given a dataset that consists of a set of ratings produced by some users on some movies, 
you wish to learn the parameter vectors $ x^{(1)}, \ldots, x^{(n_m)}, \theta^{(1)}, \ldots, \theta^{(n_u)} $ 
that produce the best fit (minimizes the squared error).

The `cofiCostFunc` computes the cost function and gradient for collaborative filtering. 
Note that the parameters to the function (i.e., the values that you are trying to learn) are `X` and `Theta`. 
In order to use an off-the-shelf minimizer such as `scipy.minimize`, 
the cost function has been set up to unroll the parameters into a single vector `params`. 

### Collaborative Filtering Cost Function

The collaborative filtering cost function (without regularization) is given by:

$$
J(x^{(1)}, \ldots, x^{(n_m)}, \theta^{(1)}, \ldots, \theta^{(n_u)}) = \frac{1}{2} \sum_{(i,j): r(i,j) = 1} ((\theta^{(j)})^T x^{(i)} - y^{(i,j)})^2.
$$

Note that we only accumulate the cost for user $ j $ and movie $ i $ if $ R(i,j) = 1 $.

Finally, we run this cost function. We expect to see an output of 22.22 when we use a reduced dataset `(num_users = 4;num_movies = 5;num_features = 3)`


In [4]:
movieParams = scipy.io.loadmat('./data/ex8_movieParams.mat')
movieParams.keys()

dict_keys(['__header__', '__version__', '__globals__', 'X', 'Theta', 'num_users', 'num_movies', 'num_features'])

In [5]:
X, Theta = movieParams['X'], movieParams['Theta']
X.shape, Theta.shape

((1682, 10), (943, 10))

In [6]:
num_users, num_movies, num_features = movieParams['num_users'], movieParams['num_movies'], movieParams['num_features']
num_users, num_movies, num_features = num_users[0][0], num_movies[0][0], num_features[0][0]
num_users, num_movies, num_features

(943, 1682, 10)

In [25]:
def J(X, Theta, Y, R):
    return np.sum(((X.dot(Theta.T) - Y) * R)**2)/2

In [30]:
# Reduce the data set size so that this runs faster
num_users = 4
num_movies = 5
num_features = 3
X1 = X[0:num_movies, 0:num_features]
Theta1 = Theta[0:num_users, 0:num_features]
Y1 = Y[0:num_movies, 0:num_users]
R1 = R[0:num_movies, 0:num_users]
X1.shape, Theta1.shape, Y1.shape, R1.shape

((5, 3), (4, 3), (5, 4), (5, 4))

In [32]:
J(X1, Theta1, Y1, R1)

22.224603725685675

In [ ]:
scipy.optimize.minimize(fun=J, method='BFGS')

In [34]:
np.vstack((X, Theta)).shape

(2625, 10)